In [ ]:
import arcpy
import requests
import os
import zipfile
import io
import sys

In [ ]:
#Dory Starting Point
arcpy.management.XYTableToPoint(r"C:\Users\siyal\Desktop\UMN MGIS\1st Semester\2. ArcGIS I\2. Labs\Lab 2\Part 2\Starting & End Points\Dory Starting Point.csv", r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\DoryStartingPoint_XYTableToPoint", "Long", "Lat", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

In [ ]:
#Dory Ending Point
arcpy.management.XYTableToPoint(r"C:\Users\siyal\Desktop\UMN MGIS\1st Semester\2. ArcGIS I\2. Labs\Lab 2\Part 2\Starting & End Points\Dory Ending Point.csv", r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\DoryEndingPoint_XYTableToPoint1", "Long", "Lat", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

# Select Layer by Attribute
Study Extent (3 Counties)

In [ ]:
arcpy.management.SelectLayerByAttribute("mn_county_boundaries_multipart", "NEW_SELECTION", "COUNTYNAME = 'Wabasha' Or COUNTYNAME = 'Winona' Or COUNTYNAME = 'Olmsted'", None)

# Streams in the study extent 
Using Clip Tool (Vector Data) 

In [ ]:
arcpy.analysis.Clip("streams_with_strahler_stream_order", "mn_county_boundarie_Dissolve2", r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\streams_with_strahler_s_Clip", None)

# Feature to Raster (Streams)

In [ ]:
arcpy.conversion.FeatureToRaster("streams_with_strahler_s_Clip", "FW_ID", r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Feature_stre1", 276.583641600005)

# DEM in the study extent 
Using Extract by Mask Tool (Raster Data) 

In [ ]:
out_raster = arcpy.sa.ExtractByMask("digital_elevation_model_30m", "mn_county_boundarie_Dissolve2", "INSIDE", '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); out_raster.save(r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Extract_digi1")

# DEM percentage rise using slope tool

In [ ]:
arcpy.ddd.Slope("DEM_Study_Extent", r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Slope_Extrac1", "PERCENT_RISE", 1, "PLANAR", "METER")

# Roads in the study extent
Using Extract by Mask Tool (Raster Data)

In [ ]:
out_raster = arcpy.sa.ExtractByMask("NLCD 2019 - Imperviousness", "mn_county_boundarie_Dissolve2", "INSIDE", '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); out_raster.save(r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Extract_NLCD1")

# Land Cover & Land Use in the study extent
Using Extract by Mask Tool (Raster Data)

In [ ]:
out_raster = arcpy.sa.ExtractByMask("NLCD_2019_Land_Cover.tif", "mn_county_boundarie_Dissolve2", "INSIDE", '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); out_raster.save(r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Extract_NLCD2")

# Reclassification of Input Layers to Standardized scale

# Streams Layer
Reclassify the layer by assigning preferences (1-Highest ; 9-Lowest)

In [ ]:
arcpy.ddd.Reclassify("Streams Raster Layer", "Value", "0 13757 1;13757 27514 2;27514 41271 3;41271 55028 4;55028 68785 5;68785 82542 6;82542 96299 7;96299 110056 8;110056 123813 9;NODATA 1", r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Reclass_Feat4", "DATA")

# DEM Data
Reclassify the data by assigning preferences (1-Highest ; 9-Lowest)

In [ ]:
arcpy.ddd.Reclassify("Slope_Extrac1", "VALUE", "0 3 1;3 6 2;6 10 3;10 15 4;15 20 5;20 25 5;25 30 6;30 40 6;40 60 7;60 100 8;100 1000 9", r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Reclass_Slop4", "DATA")

# Road Data
Reclassify the data by assigning preferences (9-Highest ; 1-Lowest)

In [ ]:
arcpy.ddd.Reclassify("Roads_Study_Extent", "Value", "0 9 9;9 28 7;28 49 5;49 72 3;72 100 1", r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Reclass_Extr3", "DATA")

# Land Cover
Reclassify the data by assigning preferences 

In [ ]:
arcpy.ddd.Reclassify("Land_Cover", "NLCD_Land", "'Open Water' 9;'Developed, Open Space' 1;'Developed, Low Intensity' 3;'Developed, Medium Intensity' 3;'Developed, High Intensity' 4;'Barren Land' 1;'Deciduous Forest' 7;'Evergreen Forest' 8;'Mixed Forest' 7;Shrub/Scrub 5;Herbaceous 7;Hay/Pasture 9;'Cultivated Crops' 9;'Woody Wetlands' 9;'Emergent Herbaceous Wetlands' 8", r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Reclass_Extr5", "DATA")

# Weighted Overlay 
Assigning equal weights to all inputs

In [ ]:
out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Reclass_Slop4' 25 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Reclass_Extr3' 25 'Value' (1 1; 3 3; 5 5; 7 7; 9 9; NODATA NODATA); 'C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Reclass_Extr5' 25 'Value' (1 1; 3 3; 4 4; 5 5; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Extract_Recl2' 25 'Value' (1 1; 8 8; 9 9; NODATA NODATA));1 9 1"); out_raster.save(r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\Weighte_Recl2")

# Cost Distance Tool

In [ ]:
out_distance_raster = arcpy.sa.CostDistance("DoryStartingPoint_XYTableToPoint", "Weighted_Overlay", None, r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\cost_direction", None, None, None, None, ''); out_distance_raster.save(r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\CostDis_Dory1")

# Cost Path As Polyline Tool

In [ ]:
arcpy.sa.CostPathAsPolyline("DoryEndingPoint_XYTableToPoint1", "CostDis_Dory1", "cost_direction", r"C:\Users\siyal\Documents\ArcGIS\Dora_ArcGIS_Lab02\MyProject3\MyProject3.gdb\CostPat_DoryEnd1", "BEST_SINGLE", "OBJECTID", "INPUT_RANGE")